<a href="https://colab.research.google.com/github/katiasioufii/Bones_fracrion/blob/main/Bones_Fraction_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 😊Bones Fraction Detection Project

###Importing the required Libraries 📚

In [ ]:
from google.colab import drive
!pip install opencv-python
import os
import cv2
import numpy as np
!pip install ultralytics
from ultralytics import YOLO
from tqdm import tqdm
import shutil
import albumentations as A
from PIL import Image
import pandas as pd

###Download the Data

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dataset_path = '/content/drive/MyDrive/bonesfracrion/bone fracture detection.v4-v4.yolov8/'


In [ ]:
print(os.listdir(dataset_path))
print(os.listdir(os.path.join(dataset_path, 'train')))
print(os.listdir(os.path.join(dataset_path, 'test')))
print(os.listdir(os.path.join(dataset_path, 'valid')))

['README.dataset.txt', 'data.yaml', 'train', 'valid', 'test']
['labels', 'images']
['labels', 'images']
['labels', 'images']


###Image preproccing

In [ ]:
def preprocess_image(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)


    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    img = clahe.apply(img)

    # Gaussian Blur
    img = cv2.GaussianBlur(img, (5,5), 0)

    edges = cv2.Canny(img, 100, 200)
    img = cv2.addWeighted(img, 0.8, edges, 0.2, 0)

    img = img / 255.0
    return (img * 255).astype(np.uint8)

In [ ]:
for split in ["train", "valid", "test"]:
    os.makedirs(f"{processed_dataset_path}/{split}/images", exist_ok=True)
    os.makedirs(f"{processed_dataset_path}/{split}/labels", exist_ok=True)

In [16]:
processed_dataset_path = "BoneFracture_Preprocessed"

for split in ["train", "valid", "test"]:
    images_dir = f"{dataset_path}/{split}/images"
    labels_dir = f"{dataset_path}/{split}/labels"
    output_images_dir = f"{processed_dataset_path}/{split}/images"
    output_labels_dir = f"{processed_dataset_path}/{split}/labels"

    for img_file in tqdm(os.listdir(images_dir), desc=f"Processing {split} images"):
        img_path = os.path.join(images_dir, img_file)
        processed_img = preprocess_image(img_path)
        cv2.imwrite(os.path.join(output_images_dir, img_file), processed_img)

        label_file = img_file.replace(".jpg", ".txt")
        if os.path.exists(os.path.join(labels_dir, label_file)):
            shutil.copy(os.path.join(labels_dir, label_file), os.path.join(output_labels_dir, label_file))

print("✅ Preprocessing complete!")

yaml_path_fixed = "data.yaml"
yaml_content = f"""
train: {processed_dataset_path}/train/images
val: {processed_dataset_path}/valid/images
test: {processed_dataset_path}/test/images

nc: 6
names: ["Elbow Positive", "Fingers Positive", "Forearm Fracture", "Humerus Fracture", "Shoulder Fracture", "Wrist Positive"]
"""

with open(yaml_path_fixed, "w") as f:
    f.write(yaml_content)

print("✅ data.yaml created.")

Processing test images: 100%|██████████| 169/169 [00:03<00:00, 50.77it/s]

✅ Preprocessing complete!
✅ data.yaml created.


In [17]:
shutil.move("BoneFracture_Preprocessed", "/content/datasets/BoneFracture_Preprocessed")


'/content/datasets/BoneFracture_Preprocessed'

##YOLO MODEL

In [ ]:
model = YOLO("yolov8n.pt")
model.train(
    data=yaml_path_fixed,
    epochs=50,
    imgsz=640,
    batch=8,
    workers=4,
    project="yolo_training",
    name="fracture_detector_v1",
    verbose=True
)

import time
for i in range(120):
    print(f"⚡ Training in progress... heartbeat {i+1}")
    time.sleep(240)



test_data_path = "/kaggle/input/test-data"
results = model.predict(source=test_data_path, save=True, save_txt=True, conf=0.25)

100%|██████████| 6.25M/6.25M [00:00<00:00, 56.9MB/s]


Ultralytics 8.3.130 🚀 Python-3.11.12 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=fracture_detector_v1, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=True, pr

100%|██████████| 755k/755k [00:00<00:00, 24.6MB/s]

Overriding model.yaml nc=80 with nc=6

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           


  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  8                  -1  1    460288  ultralytics.nn.modules.block.C2f             [256, 256, 1, True]           
  9                  -1  1    164608  ultralytics.nn.modules.block.SPPF            [256, 256, 5]                 
 10                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 11             [-1, 6]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 12                  -1  1    148224  ultralytics.nn.modules.block.C2f             [384, 128, 1]                 
 13                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 14             [-1, 4]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 15                  -1  1     37248  ultralytics.nn.modules.block.C2f             [192,

...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       2/50         0G      2.633      4.921      2.176          5        640: 100%|██████████| 436/436 [47:04<00:00,  6.48s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 21/21 [01:27<00:00,  4.17s/it]

                   all        331        176       0.65     0.0204     0.0147    0.00404



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       3/50         0G      2.584        4.3      2.129         11        640:  15%|█▌        | 66/436 [07:10<39:42,  6.44s/it]



```
# This is formatted as code
```



In [ ]:

# Function to draw bounding boxes
def draw_bounding_boxes(image_path, label_path):
    # Read the image
    image = cv2.imread(image_path)
    height, width, _ = image.shape

    # Open the label file
    with open(label_path, 'r') as f:
        lines = f.readlines()

    # Draw each bounding box
    for line in lines:
        # Each line in the label file corresponds to a bounding box
        parts = line.strip().split()
        class_id = int(parts[0])  # class_id (not really needed, but can be useful)
        x_center = float(parts[1])
        y_center = float(parts[2])
        box_width = float(parts[3])
        box_height = float(parts[4])

        # Convert normalized coordinates to pixel values
        x_min = int((x_center - box_width / 2) * width)
        y_min = int((y_center - box_height / 2) * height)
        x_max = int((x_center + box_width / 2) * width)
        y_max = int((y_center + box_height / 2) * height)

        # Draw the bounding box on the image
        color = (0, 255, 0)  # Green color for the box
        thickness = 2  # Thickness of the bounding box
        image = cv2.rectangle(image, (x_min, y_min), (x_max, y_max), color, thickness)

    # Save or show the image with annotations
    annotated_image_path = image_path.replace("img", "annotated_img")  # Save to annotated_img folder
    os.makedirs(os.path.dirname(annotated_image_path), exist_ok=True)
    cv2.imwrite(annotated_image_path, image)
    cv2.imshow("Annotated Image", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Directory paths (adjust as needed)
image_folder = dataset_path+'train/images'  # e.g.
label_folder = dataset_path+'train/labels' # e.g.

for img_filename in os.listdir(image_folder):
    if img_filename.endswith('.jpg') or img_filename.endswith('.png'):
        image_path = os.path.join(image_folder, img_filename)
        label_path = os.path.join(label_folder, img_filename.replace('.jpg', '.txt').replace('.png', '.txt'))

        # Check if the label file exists
        if os.path.exists(label_path):
            draw_bounding_boxes(image_path, label_path)

print("Annotation complete!")

DisabledFunctionError: cv2.imshow() is disabled in Colab, because it causes Jupyter sessions
to crash; see https://github.com/jupyter/notebook/issues/3935.
As a substitution, consider using
  from google.colab.patches import cv2_imshow


In [ ]:
import tensorflow as tf

def load_and_preprocess_image(path, label):
    img = tf.io.read_file(path)
    img = tf.image.decode_image(img)
    img = tf.image.resize(img, [64, 64])
    img = img / 255.0
    return img, label

dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))
dataset = dataset.map(load_and_preprocess_image)
dataset = dataset.batch(32).prefetch(tf.data.AUTOTUNE)


NameError: name 'image_paths' is not defined

In [ ]:
import tensorflow as tf
import os
import os
def load_data(data_dir):
    images = []
    labels = []

    for label_file in os.listdir(data_dir + '/labels'):
        img_filename = label_file.replace('.txt', '.jpg')
        img_path = os.path.join(data_dir, 'images', img_filename)

        if os.path.exists(img_path):
            with open(os.path.join(data_dir, 'labels', label_file), 'r') as f:
                label_data = f.readlines()

            bbox_list = []
            for line in label_data:
                values = list(map(float, line.strip().split()))
                bbox_list.append(values)  # Assuming they are in [class, x_center, y_center, width, height]

            images.append(tf.io.read_file(img_path))
            # Ensure the bounding boxes are in the correct format
            labels.append(tf.convert_to_tensor(bbox_list, dtype=tf.float32))
        else:
            print(f"Image not found: {img_path}")

    return images, labels

In [ ]:

def preprocess_image(image):
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])
    image = image / 255.0
    return image

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))

def map_func(image, label):
    return preprocess_image(image), label  # Keep label as tensor

train_dataset = train_dataset.map(map_func).batch(32).prefetch(tf.data.AUTOTUNE)

In [ ]:
import tensorflow as tf

# Define the model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    # Output layer for bounding boxes [x_center, y_center, width, height]
    tf.keras.layers.Dense(4, activation='sigmoid')  # Adjust for your output
])

In [ ]:

def custom_loss(y_true, y_pred):
    # Adjust for multiple bounding boxes
    return tf.reduce_mean(tf.square(y_true - y_pred), axis=-1)  # Calculate loss per bounding box

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))

def map_func(image, label):
    return preprocess_image(image), tf.convert_to_tensor(label, dtype=tf.float32)

train_dataset = train_dataset.map(map_func).batch(32).prefetch(tf.data.AUTOTUNE)

model.compile(optimizer='adam', loss=custom_loss)
model.fit(train_dataset, epochs=10)

Epoch 1/10


ValueError: Dimensions must be equal, but are 0 and 4 for '{{node compile_loss/custom_loss/sub}} = Sub[T=DT_FLOAT](data_1, sequential_2_1/dense_6_1/Sigmoid)' with input shapes: [?,0], [?,4].

In [ ]:
num_boxes = 5  # Example: Adjust this based on your dataset

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    # Output layer for multiple bounding boxes
    tf.keras.layers.Dense(num_boxes * 4, activation='sigmoid'),  # Flattened output
    tf.keras.layers.Reshape((num_boxes, 4))  # Reshape to (num_boxes, 4)
])

In [ ]:
def load_data(data_dir):
    images = []
    labels = []

    for label_file in os.listdir(data_dir + '/labels'):
        img_filename = label_file.replace('.txt', '.jpg')
        img_path = os.path.join(data_dir, 'images', img_filename)

        if os.path.exists(img_path):
            with open(os.path.join(data_dir, 'labels', label_file), 'r') as f:
                label_data = f.readlines()

            bbox_list = []
            for line in label_data:
                values = list(map(float, line.strip().split()))
                bbox_list.append(values)  # Assuming they are in [x_center, y_center, width, height]

            # Pad or truncate the bbox_list to match num_boxes
            while len(bbox_list) < num_boxes:
                bbox_list.append([0, 0, 0, 0])  # Padding with zeros
            bbox_list = bbox_list[:num_boxes]  # Truncate if necessary

            images.append(tf.io.read_file(img_path))
            labels.append(tf.convert_to_tensor(bbox_list, dtype=tf.float32))
        else:
            print(f"Image not found: {img_path}")

    return images, labels

In [ ]:
model.compile(optimizer='adam', loss=custom_loss)

# Fit the model to the dataset
model.fit(train_dataset, epochs=10)  # Adjust the number of epochs as needed

Epoch 1/10


ValueError: Dimensions must be equal, but are 0 and 4 for '{{node compile_loss/custom_loss/sub}} = Sub[T=DT_FLOAT](data_1, sequential_3_1/reshape_1/Reshape)' with input shapes: [?,0], [?,5,4].

In [ ]:
import tensorflow as tf
import os

# Constants
IMAGE_HEIGHT, IMAGE_WIDTH = 224, 224
NUM_BOXES = 5  # Adjust based on your dataset

# Data loading function
def load_data(data_dir):
    images = []
    labels = []

    for label_file in os.listdir(os.path.join(data_dir, 'labels')):
        img_filename = label_file.replace('.txt', '.jpg')
        img_path = os.path.join(data_dir, 'images', img_filename)

        if os.path.exists(img_path):
            with open(os.path.join(data_dir, 'labels', label_file), 'r') as f:
                label_data = f.readlines()

            bbox_list = []
            for line in label_data:
                values = list(map(float, line.strip().split()))
                bbox_list.append(values)  # Assuming [x_center, y_center, width, height]

            # Pad or truncate to fit NUM_BOXES
            while len(bbox_list) < NUM_BOXES:
                bbox_list.append([0, 0, 0, 0])  # Padding
            bbox_list = bbox_list[:NUM_BOXES]  # Truncate if necessary

            images.append(tf.io.read_file(img_path))
            labels.append(tf.convert_to_tensor(bbox_list, dtype=tf.float32))
        else:
            print(f"Image not found: {img_path}")

    return images, labels

# Preprocessing function (customize as needed)
def preprocess_image(image):
    img = tf.image.decode_jpeg(image, channels=3)
    img = tf.image.resize(img, [IMAGE_HEIGHT, IMAGE_WIDTH])
    img = img / 255.0  # Normalize to [0, 1]
    return img

# Custom loss function
def custom_loss(y_true, y_pred):
    return tf.reduce_mean(tf.square(y_true - y_pred), axis=[1, 2])  # Average over boxes and batches

# Model definition
def create_model():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_HEIGHT, IMAGE_WIDTH, 3)),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(NUM_BOXES * 4, activation='sigmoid'),  # Output layer
        tf.keras.layers.Reshape((NUM_BOXES, 4))  # Reshape to (NUM_BOXES, 4)
    ])
    return model

# Main execution
if __name__ == "__main__":
    data_dir = dataset_path+'train/'  # Update this to your data directory
    train_images, train_labels = load_data(data_dir)

    # Prepare dataset
    train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
    train_dataset = train_dataset.map(lambda img, lbl: (preprocess_image(img), lbl)).batch(32).prefetch(tf.data.AUTOTUNE)

    # Create and compile the model
    model = create_model()
    model.compile(optimizer='adam', loss=custom_loss)

    # Train the model
    model.fit(train_dataset, epochs=10)

    # Save the model
    model.save('my_object_detection_model.h5')

FileNotFoundError: [Errno 2] No such file or directory: 'path/to/your/data/labels'

In [ ]:
import tensorflow as tf
import os

# Constants
IMAGE_HEIGHT, IMAGE_WIDTH = 224, 224
NUM_BOXES = 5  # Adjust based on your dataset

# Data loading function
def load_data(data_dir):
    images = []
    labels = []

    for label_file in os.listdir(os.path.join(data_dir, 'labels')):
        img_filename = label_file.replace('.txt', '.jpg')
        img_path = os.path.join(data_dir, 'images', img_filename)

        if os.path.exists(img_path):
            with open(os.path.join(data_dir, 'labels', label_file), 'r') as f:
                label_data = f.readlines()

            bbox_list = []
            for line in label_data:
                values = list(map(float, line.strip().split()))
                bbox_list.append(values)


            while len(bbox_list) < NUM_BOXES:
                bbox_list.append([0, 0, 0, 0])  # Padding with zeros
            bbox_list = bbox_list[:NUM_BOXES]  # Truncate if necessary

            images.append(tf.io.read_file(img_path))
            labels.append(bbox_list)
        else:
            print(f"Image not found: {img_path}")


    images = tf.convert_to_tensor(images, dtype=tf.string)  # Images as string tensors
    labels = tf.convert_to_tensor(labels, dtype=tf.float32)  # Labels as float tensors

    return images, labels

# Preprocessing function (customize as needed)
def preprocess_image(image):
    img = tf.image.decode_jpeg(image, channels=3)
    img = tf.image.resize(img, [IMAGE_HEIGHT, IMAGE_WIDTH])
    img = img / 255.0  # Normalize to [0, 1]
    return img

# Custom loss function
def custom_loss(y_true, y_pred):
    return tf.reduce_mean(tf.square(y_true - y_pred), axis=[1, 2])  # Average over boxes and batches

# Model definition
def create_model():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_HEIGHT, IMAGE_WIDTH, 3)),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(NUM_BOXES * 4, activation='sigmoid'),  # Output layer
        tf.keras.layers.Reshape((NUM_BOXES, 4))  # Reshape to (NUM_BOXES, 4)
    ])
    return model

# Main execution
if __name__ == "__main__":
    data_dir = dataset_path+'train/'  # Update this to your data directory
    train_images, train_labels = load_data(data_dir)

    # Prepare dataset
    train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
    train_dataset = train_dataset.map(lambda img, lbl: (preprocess_image(img), lbl)).batch(32).prefetch(tf.data.AUTOTUNE)

    # Create and compile the model
    model = create_model()
    model.compile(optimizer='adam', loss=custom_loss)

    # Train the model
    model.fit(train_dataset, epochs=10)

    # Save the model
    model.save('my_object_detection_model.h5')

Image not found: /content/drive/MyDrive/bonesfracrion/bone fracture detection.v4-v4.yolov8/train/images/image1_3123_png.rf.cc421f7578cef1c5eae31ed403f3350b (1).jpg
Image not found: /content/drive/MyDrive/bonesfracrion/bone fracture detection.v4-v4.yolov8/train/images/image1_191_png.rf.4281c0740362e3f7870d931d1e405aab (1).jpg
Image not found: /content/drive/MyDrive/bonesfracrion/bone fracture detection.v4-v4.yolov8/train/images/image1_309_png.rf.acab65f10f04644c5e615777d54e4fa2 (1).jpg
Image not found: /content/drive/MyDrive/bonesfracrion/bone fracture detection.v4-v4.yolov8/train/images/image1_190_png.rf.f91841e6b0b9082579f5cbcfd290182d (1).jpg
Image not found: /content/drive/MyDrive/bonesfracrion/bone fracture detection.v4-v4.yolov8/train/images/image1_1932_png.rf.bc30a0a88edc82de1d4cdbdbfcb42522 (1).jpg
Image not found: /content/drive/MyDrive/bonesfracrion/bone fracture detection.v4-v4.yolov8/train/images/image1_30_png.rf.365839c48a61141f1c1c96ccc9c02fa3 (1).jpg
Image not found: /con

ValueError: Can't convert non-rectangular Python sequence to Tensor.